In [7]:
import pandas as pd

import pandas as pd
combined_df=pd.read_csv("input-2.csv")



In [8]:
combined_df

,text,class
0,news latest headlines on cnn business n tldr t...,ctrl
1,news china wants to take a victory lap over it...,ctrl
2,news coronavirus disinformation creates challe...,ctrl
3,news china coronavirus eating wild animals mad...,ctrl
4,news chinas economy could shrink for the first...,ctrl
...,...,...
11721,how much of your body is your own how much of ...,instructgpt
11722,how do you keep a space station clean how do y...,instructgpt
11723,the city where you pay a years rent up front t...,instructgpt
11724,the bbc news app gives you the best of bbc new...,instructgpt


In [9]:
import pandas as pd


human_data = combined_df[combined_df['class'] == 'human']


classes = ['ctrl', 'fair', 'gpt', 'gpt2', 'gpt3', 'grover', 'instructgpt', 'pplm', 'xlm', 'xlnet']


class_dataframes = []
for class_name in classes:
    class_data = combined_df[combined_df['class'] == class_name]
    class_dataframe = pd.concat([human_data, class_data]).reset_index(drop=True)
    class_dataframes.append(class_dataframe)

for i, class_name in enumerate(classes):
    print(f"Data frame for class {class_name}:")
    print(class_dataframes[i])
    print()


Data frame for class ctrl:
                                                   text  class
0     latest headlines on cnn business latest headli...  human
1     china wants to take a victory lap over its han...  human
2     coronavirus disinformation creates challenges ...  human
3     china coronavirus eating wild animals made ill...  human
4     chinas economy could shrink for the first time...  human
...                                                 ...    ...
2127  news how much of your body is your own n a i t...   ctrl
2128  news how do you keep a space station clean n q...   ctrl
2129  news the city where you pay a years rent up fr...   ctrl
2130  news the bbc news app gives you the best of bb...   ctrl
2131  news learn how the bbc is working to strengthe...   ctrl

[2132 rows x 2 columns]

Data frame for class fair:
                                                   text  class
0     latest headlines on cnn business latest headli...  human
1     china wants to take a victory la

In [10]:
df=class_dataframes[0]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
%pip install transformers
from transformers import BertForSequenceClassification, BertTokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
import torch

def tokenize_and_encode(text, tokenizer, max_length):
    input_ids = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_length, padding='max_length', truncation=True, return_token_type_ids=False, return_attention_mask=True, return_tensors='pt')
    return input_ids['input_ids'], input_ids['attention_mask']

In [13]:
def get_dataloader(df, tokenizer, max_length, batch_size, shuffle=True):
    texts = df['text'].values
    labels = df['class'].values
    
    input_ids = []
    attention_masks = []
    
    for text in texts:
        input_id, attention_mask = tokenize_and_encode(text, tokenizer, max_length)
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    
    dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [14]:
def train_model(model, train_dataloader, optimizer, device):
    model.train()
    
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        loss.backward()
        optimizer.step()
    
    return model

In [15]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

def evaluate_model(model, val_dataloader, device):
    model.eval()
    val_loss, val_accuracy = 0, 0
    n_val_steps = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_masks, labels = batch

            outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            preds = logits.argmax(dim=1)
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())
            val_accuracy += (preds == labels).float().mean().item()
            n_val_steps += 1

    val_loss /= n_val_steps
    val_accuracy /= n_val_steps

    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    classification_rep = classification_report(all_labels, all_preds)

    return val_loss, val_accuracy, precision, recall, f1, classification_rep


In [16]:
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy)  
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS CTRL(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS CTRL(0)
Epoch 1 Val Loss: 0.005, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

HUMAN(1)  VS CTRL(0)
Epoch 2 Val Loss: 0.001, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

HUMAN(1)  VS CTRL(0)
Epoch 3 Val Loss: 0.001, Val Accu

In [ ]:
df=class_dataframes[1]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy) 
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS fair(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS fair(0)
Epoch 1 Val Loss: 0.397, Val Accuracy: 0.835, Val Precision: 0.862, Val Recall: 0.836, Val F1: 0.833
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.97      0.86       217
           1       0.96      0.70      0.81       210

    accuracy                           0.84       427
   macro avg       0.86      0.83      0.83       427
weighted avg       0.86      0.84      0.83       427

HUMAN(1)  VS fair(0)
Epoch 2 Val Loss: 0.290, Val Accuracy: 0.873, Val Precision: 0.874, Val Recall: 0.874, Val F1: 0.874
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.88      0.88       217
           1       0.87      0.87      0.87       210

    accuracy                           0.87       427
   macro avg       0.87      0.87      0.87       427
weighted avg       0.87      0.87      0.87       427

HUMAN(1)  VS fair(0)
Epoch 3 Val Loss: 0.352, Val Accu

In [ ]:
df=class_dataframes[2]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy) 
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS gpt(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS gpt(0)
Epoch 1 Val Loss: 0.023, Val Accuracy: 0.993, Val Precision: 0.993, Val Recall: 0.993, Val F1: 0.993
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       217
           1       1.00      0.99      0.99       210

    accuracy                           0.99       427
   macro avg       0.99      0.99      0.99       427
weighted avg       0.99      0.99      0.99       427

HUMAN(1)  VS gpt(0)
Epoch 2 Val Loss: 0.004, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

HUMAN(1)  VS gpt(0)
Epoch 3 Val Loss: 0.002, Val Accurac

In [17]:
df=class_dataframes[3]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy)  
    val_accuracies.append(val_accuracy) 
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS gpt2(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS gpt2(0)
Epoch 1 Val Loss: 0.027, Val Accuracy: 0.991, Val Precision: 0.991, Val Recall: 0.991, Val F1: 0.991
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       217
           1       0.98      1.00      0.99       210

    accuracy                           0.99       427
   macro avg       0.99      0.99      0.99       427
weighted avg       0.99      0.99      0.99       427

HUMAN(1)  VS gpt2(0)
Epoch 2 Val Loss: 0.014, Val Accuracy: 0.998, Val Precision: 0.998, Val Recall: 0.998, Val F1: 0.998
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

HUMAN(1)  VS gpt2(0)
Epoch 3 Val Loss: 0.009, Val Accu

In [ ]:
df=class_dataframes[4]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy)  
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS gpt3(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS gpt3(0)
Epoch 1 Val Loss: 0.125, Val Accuracy: 0.962, Val Precision: 0.964, Val Recall: 0.963, Val F1: 0.962
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.99      0.96       217
           1       0.99      0.93      0.96       210

    accuracy                           0.96       427
   macro avg       0.96      0.96      0.96       427
weighted avg       0.96      0.96      0.96       427

HUMAN(1)  VS gpt3(0)
Epoch 2 Val Loss: 0.088, Val Accuracy: 0.975, Val Precision: 0.974, Val Recall: 0.974, Val F1: 0.974
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.97       217
           1       0.98      0.97      0.97       210

    accuracy                           0.97       427
   macro avg       0.97      0.97      0.97       427
weighted avg       0.97      0.97      0.97       427

HUMAN(1)  VS gpt3(0)
Epoch 3 Val Loss: 0.144, Val Accu

In [18]:
df=class_dataframes[5]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy)  
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS grover(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS grover(0)
Epoch 1 Val Loss: 0.103, Val Accuracy: 0.968, Val Precision: 0.968, Val Recall: 0.967, Val F1: 0.967
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.94      0.97       217
           1       0.95      0.99      0.97       210

    accuracy                           0.97       427
   macro avg       0.97      0.97      0.97       427
weighted avg       0.97      0.97      0.97       427

HUMAN(1)  VS grover(0)
Epoch 2 Val Loss: 0.085, Val Accuracy: 0.968, Val Precision: 0.969, Val Recall: 0.967, Val F1: 0.967
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97       217
           1       0.94      1.00      0.97       210

    accuracy                           0.97       427
   macro avg       0.97      0.97      0.97       427
weighted avg       0.97      0.97      0.97       427

HUMAN(1)  VS grover(0)
Epoch 3 Val Loss: 0.112, Va

In [ ]:
df=class_dataframes[6]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy) 
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS InstructGPT(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS InstructGPT(0)
Epoch 1 Val Loss: 0.105, Val Accuracy: 0.963, Val Precision: 0.963, Val Recall: 0.963, Val F1: 0.963
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.98      0.96       217
           1       0.98      0.95      0.96       210

    accuracy                           0.96       427
   macro avg       0.96      0.96      0.96       427
weighted avg       0.96      0.96      0.96       427

HUMAN(1)  VS InstructGPT(0)
Epoch 2 Val Loss: 0.103, Val Accuracy: 0.970, Val Precision: 0.971, Val Recall: 0.970, Val F1: 0.970
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       217
           1       1.00      0.94      0.97       210

    accuracy                           0.97       427
   macro avg       0.97      0.97      0.97       427
weighted avg       0.97      0.97      0.97       427

HUMAN(1)  VS InstructGPT(0)
Epoch 3 Val 

In [19]:
df=class_dataframes[7]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy)  
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS PPLM(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS PPLM(0)
Epoch 1 Val Loss: 0.010, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

HUMAN(1)  VS PPLM(0)
Epoch 2 Val Loss: 0.002, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

HUMAN(1)  VS PPLM(0)
Epoch 3 Val Loss: 0.001, Val Accu

In [20]:
df=class_dataframes[8]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy)  
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1 Val Loss: 0.004, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

Epoch 2 Val Loss: 0.001, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

Epoch 3 Val Loss: 0.001, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1

In [21]:
df=class_dataframes[9]
df['class'] = df['class'].apply(lambda x: 1 if x=="human" else 0)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch
import matplotlib.pyplot as plt

MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

train_accuracies = []
val_accuracies = []
f1_scores = []

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy, val_precision, val_recall, val_f1, classification_rep = evaluate_model(model, val_dataloader, device)
    train_accuracies.append(val_accuracy)  
    val_accuracies.append(val_accuracy)  
    f1_scores.append(val_f1)  
    print("HUMAN(1)  VS XLNET(0)")
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}, Val Precision: {val_precision:.3f}, Val Recall: {val_recall:.3f}, Val F1: {val_f1:.3f}")
    print("Classification Report:\n", classification_rep)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

HUMAN(1)  VS XLNET(0)
Epoch 1 Val Loss: 0.005, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

HUMAN(1)  VS XLNET(0)
Epoch 2 Val Loss: 0.002, Val Accuracy: 1.000, Val Precision: 1.000, Val Recall: 1.000, Val F1: 1.000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       217
           1       1.00      1.00      1.00       210

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427

HUMAN(1)  VS XLNET(0)
Epoch 3 Val Loss: 0.001, Val A

In [ ]:
unseen_df = df1  

unseen_dataloader = get_dataloader(unseen_df, tokenizer, MAX_LENGTH, BATCH_SIZE)


unseen_loss, unseen_accuracy, unseen_precision, unseen_recall, unseen_f1, unseen_classification_rep = evaluate_model(model, unseen_dataloader, device)


print("Classification Report for REDDIt unseen Data:")
print(unseen_classification_rep)
print(f"Accuracy: {unseen_accuracy:.3f}")
print(f"Precision: {unseen_precision:.3f}")
print(f"Recall: {unseen_recall:.3f}")
print(f"F1-Score: {unseen_f1:.3f}")


Classification Report for REDDIt unseen Data:
              precision    recall  f1-score   support

           0       0.65      0.94      0.77      1224
           1       0.39      0.07      0.11       652

    accuracy                           0.64      1876
   macro avg       0.52      0.51      0.44      1876
weighted avg       0.56      0.64      0.54      1876

Accuracy: 0.637
Precision: 0.562
Recall: 0.639
F1-Score: 0.544
